In [2]:
# Some imports
import sys
import os, glob
from mfx.processlocalizations import ProcessLocalizations
import re
from matplotlib import pyplot as plt
import math
import numpy as np
from mfx import mfxcolnames as col

In [16]:
# Get file names

def gen_directories( maindir, subdir, keys):
    outdir = {}
    for key in keys:
        outdir[key] = os.path.join(maindir, subdir, key)
    return outdir

def gen_npyfiles(dir_dict):
    npy_files = {}
    for key in dir_dict:
        npy_files[key] = []

    for key in dir_dict:
        for (root, dirs, files) in os.walk(dir_dict[key]):
            if len(dirs) > 0:
                for adir in dirs:
                    npyfile = os.path.join(root, adir, adir +'.npy')
                    if os.path.exists(npyfile):
                        npy_files[key].append(npyfile)
                    
    return npy_files


OUTDIR_LOC = 'C:/Users/apoliti/Desktop/mflux_zarr_tmp_storage/analysis' # Main directory to store zarr files
OUTDIR_REM =  'Z:/siva_minflux/analysis'  # Main directory to store results 
INDIR  = 'Z:/siva_minflux/data'       # main directory of msr file

# Multiple washes with different imager strand
keys = ['Syp_ATG9', 'ZnT3_Syp', 'Syp_Picc']
indir_mwash = gen_directories(INDIR, 'Multiwash', keys)
outdir_mwash = gen_directories(OUTDIR_REM, 'Multiwash', keys)
zarrdir_mwash = gen_directories(OUTDIR_LOC, 'Multiwash', keys) 
npy_mwash = gen_npyfiles(outdir_mwash)

# Wash with a single imager strand
keys = ['Syp', 'ATG9', 'VGLUT1']
indir_swash =  gen_directories(INDIR, 'Single wash', keys)
outdir_swash =  gen_directories(OUTDIR_REM, 'Single wash', keys)
zarrdir_swash =  gen_directories(OUTDIR_LOC, 'Single wash', keys)
npy_swash = gen_npyfiles(outdir_swash)


# Consitency controls. Wash with a single imager strand but multiple times. 
keys = ['VGLUT1_VGLUT1']
indir_cwash =  gen_directories(INDIR, 'Multiwash', keys)
outdir_cwash = gen_directories(OUTDIR_REM, 'Multiwash', keys)
zarrdir_cwash = gen_directories(OUTDIR_LOC, 'Multiwash', keys) 
npy_cwash = gen_npyfiles(outdir_cwash)



Z:/siva_minflux/analysis\Multiwash\Syp_ATG9
['220510_Syp_ATG9_ROI01', '220510_Syp_ATG9_ROI02', '220510_Syp_ATG9_ROI03', '220601_Syp_ATG9_ROI01', '220601_Syp_ATG9_ROI02', '220601_Syp_ATG9_ROI03', '220601_Syp_ATG9_ROI04', '220601_Syp_ATG9_ROI05', '220601_Syp_ATG9_ROI06', '220601_Syp_ATG9_ROI07', '220614_Syp_ATG9_ROI01', '220825_Syp_ATG9_ROI01']
Z:/siva_minflux/analysis\Multiwash\ZnT3_Syp
['220309_ZnT3_Syp_ROI01', '220309_ZnT3_Syp_ROI02', '220309_ZnT3_Syp_ROI03']
Z:/siva_minflux/analysis\Multiwash\Syp_Picc
['220804_Syp_Picc_ROI01']
Z:/siva_minflux/analysis\Single wash\Syp
['220825_Syp_ROI2', '220825_Syp_ROI3', '220825_Syp_ROI4', '220825_Syp_ROI5', '220825_Syp_ROI6', 'figures']
['temp_files']
['crosstalk-1.2.0', 'htmlwidgets-1.5.4', 'jquery-3.5.1', 'plotly-binding-4.10.0', 'plotly-htmlwidgets-css-2.5.1', 'plotly-main-2.5.1', 'typedarray-0.1']
['css', 'js', 'scss']
Z:/siva_minflux/analysis\Single wash\ATG9
['220825_ATG9_ROI2', '220825_ATG9_ROI3', '220825_ATG9_ROI4', '220825_ATG9_ROI5', '220

In [18]:
def processFile(file_name, eps_range):
    pl = ProcessLocalizations(file_name)
    pl.trim_min_localizations()
    # Split tracks if needed, remove outliers in each track
    pl.cluster_tid(method=pl.CLS_METHOD_BAYES_GMM)

    # find objects that belong to the same cluster according to distance 30 nm
    for eps in eps_range:
            
        pl.DBCLUSTER_EPS_MEAS = eps
        pl.DBCLUSTER_EPS_ALL = eps
        pl.DBCLUSTER_EPS_MERGED_ALL = eps
        pl.DBCLUSTER_EPS_MERGED_MEAS = eps
        pl.log_parameters()
    
        pl.cluster_meas(method=pl.CLS_METHOD_DBSCAN)
        pl.cluster_all(method=pl.CLS_METHOD_DBSCAN)
        pl.cluster_all_intersect(col.CLS_ALL)
        summary_dict = pl.summary_per_tid2()
        pl.cluster_meas(method=pl.CLS_METHOD_DBSCAN)
        pl.cluster_all(method=pl.CLS_METHOD_DBSCAN)
        pl.cluster_all_intersect(col.CLS_ALL)
        summary_dict = pl.summary_per_tid2()
        eps_txt = '%.0f' % (eps*1e9)
        pl.export_csv(summary_dict, file_path=pl.file_path_no_ext() + '_eps' + eps_txt)
        pl.export_vtu(in_dict=summary_dict, coord=col.LTR, file_path=pl.file_path_no_ext() + '_eps' +  eps_txt)
    

In [19]:
eps_range = [1e-8, 2e-8, 3e-8, 4e-8, 5e-8, 6e-8, 7e-8, 8e-8]
for label in npy_swash:
    if label == 'VGLUT1':
        for afile in npy_swash[label]:
            processFile(afile, eps_range=eps_range)

2022-09-24 15:18:22,863 [INFO] **** trim_min_localizations ****
220621-VGLUT1_P1 Removed 610/2744 tracks with less than 3 localizations

2022-09-24 15:19:05,848 [INFO] **** cluster_tid ****
220621-VGLUT1_P1 MIN_SPLIT_LOCALIZATION: 6, sd_limit: 10.03 nm
Processed TID: 478 / 2134, Total tracks TID2: 2565

2022-09-24 15:19:05,854 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis\Single wash\VGLUT1\220621_VGLUT1_ROI01\220621_VGLUT1_ROI01.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 1e-08
DBCLUSTER_EPS_ALL 1e-08
DBCLUSTER_EPS_MERGED_ALL 1e-08
DBCLUSTER_EPS_MERGED_MEAS 1e-08
ref_error {'se_xy': 4.4834146651307424e-09, 'se_z': 2.982890853064412e-09, 'std_xy': 6.340505825170325e-09, 'std_z': 4.2184446994823425e-09}
logger <Logger processlocalizations (INFO)>

2022-09-24 15:19:06,662 [INFO] **** cluster_meas ****
220621-VGLUT1_P1, Method dbscan, eps: 10.00 nm
Total tracks TID2: 2564, total clusters meas: 1442



2022-09-24 15:19:48,131 [INFO] **** cluster_all ****
*******cluster_all*******
Only one wash. Nothing to do
2022-09-24 15:19:48,134 [INFO] **** cluster_all_intersect ****
Only one wash
export csv table
export paraview file
2022-09-24 15:19:51,256 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis\Single wash\VGLUT1\220621_VGLUT1_ROI01\220621_VGLUT1_ROI01.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 7e-08
DBCLUSTER_EPS_ALL 7e-08
DBCLUSTER_EPS_MERGED_ALL 7e-08
DBCLUSTER_EPS_MERGED_MEAS 7e-08
ref_error {'se_xy': 4.4834146651307424e-09, 'se_z': 2.982890853064412e-09, 'std_xy': 6.340505825170325e-09, 'std_z': 4.2184446994823425e-09}
logger <Logger processlocalizations (INFO)>

2022-09-24 15:19:52,978 [INFO] **** cluster_meas ****
220621-VGLUT1_P1, Method dbscan, eps: 70.00 nm
Total tracks TID2: 2564, total clusters meas: 143

2022-09-24 15:19:52,980 [INFO] **** cluster_all ****
*******cluster_all*******
Only

2022-09-24 15:23:24,028 [INFO] **** cluster_meas ****
220921-VGLUT1_P1, Method dbscan, eps: 40.00 nm
Total tracks TID2: 4880, total clusters meas: 315

2022-09-24 15:23:24,037 [INFO] **** cluster_all ****
*******cluster_all*******
Only one wash. Nothing to do
2022-09-24 15:23:24,039 [INFO] **** cluster_all_intersect ****
Only one wash
export csv table
export paraview file
2022-09-24 15:23:31,031 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis\Single wash\VGLUT1\220921_VGLUT1_ROI02\220921_VGLUT1_ROI02.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 5e-08
DBCLUSTER_EPS_ALL 5e-08
DBCLUSTER_EPS_MERGED_ALL 5e-08
DBCLUSTER_EPS_MERGED_MEAS 5e-08
ref_error {'se_xy': 1.2738087668484914e-09, 'se_z': 8.896122821272852e-10, 'std_xy': 3.120181508662668e-09, 'std_z': 2.1790961601247197e-09}
logger <Logger processlocalizations (INFO)>

2022-09-24 15:23:34,142 [INFO] **** cluster_meas ****
220921-VGLUT1_P1, Method dbsc

In [20]:
eps_range = [1e-8, 2e-8, 3e-8, 4e-8, 5e-8, 6e-8, 7e-8, 8e-8]
for label in npy_mwash:
    if label == 'Syp_Picc':
        for afile in npy_mwash[label]:
            processFile(afile, eps_range=eps_range)

2022-09-24 15:24:51,685 [INFO] **** trim_min_localizations ****
220804-Syp_P1 Removed 132/503 tracks with less than 3 localizations
220804-Picc_P2 Removed 1102/2047 tracks with less than 3 localizations

2022-09-24 15:25:07,766 [INFO] **** cluster_tid ****
220804-Syp_P1 MIN_SPLIT_LOCALIZATION: 6, sd_limit: 11.17 nm
Processed TID: 66 / 371, Total tracks TID2: 421
220804-Picc_P2 MIN_SPLIT_LOCALIZATION: 6, sd_limit: 11.17 nm
Processed TID: 205 / 945, Total tracks TID2: 1128

2022-09-24 15:25:07,768 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis\Multiwash\Syp_Picc\220804_Syp_Picc_ROI01\220804_Syp_Picc_ROI01.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 1e-08
DBCLUSTER_EPS_ALL 1e-08
DBCLUSTER_EPS_MERGED_ALL 1e-08
DBCLUSTER_EPS_MERGED_MEAS 1e-08
ref_error {'se_xy': 2.463177997334171e-09, 'se_z': 2.499140774053953e-09, 'std_xy': 4.2663494394685404e-09, 'std_z': 4.328638795928458e-09}
logger <Logger processl

2022-09-24 15:25:17,798 [INFO] **** cluster_all ****
220804-Syp_P1, Method: dbscan, eps: 40.00 nm
Total tracks TID2: 420, total cluster all: 55
220804-Picc_P2, Method: dbscan, eps: 40.00 nm
Total tracks TID2: 1127, total cluster all: 220

2022-09-24 15:25:17,854 [INFO] **** cluster_all_intersect ****
220804-Syp_P1 cluster with 220804-Picc_P2 cls 19/54 = 0.35TID2 205/420 = 0.49
220804-Picc_P2 cluster with 220804-Syp_P1 cls 19/219 = 0.09TID2 264/1127 = 0.23

export csv table
export paraview file
2022-09-24 15:25:19,045 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis\Multiwash\Syp_Picc\220804_Syp_Picc_ROI01\220804_Syp_Picc_ROI01.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 5e-08
DBCLUSTER_EPS_ALL 5e-08
DBCLUSTER_EPS_MERGED_ALL 5e-08
DBCLUSTER_EPS_MERGED_MEAS 5e-08
ref_error {'se_xy': 2.463177997334171e-09, 'se_z': 2.499140774053953e-09, 'std_xy': 4.2663494394685404e-09, 'std_z': 4.328638795928458e-09}
l

2022-09-24 15:25:32,218 [INFO] **** cluster_all ****
220804-Syp_P1, Method: dbscan, eps: 80.00 nm
Total tracks TID2: 420, total cluster all: 25
220804-Picc_P2, Method: dbscan, eps: 80.00 nm
Total tracks TID2: 1127, total cluster all: 100

2022-09-24 15:25:32,280 [INFO] **** cluster_all_intersect ****
220804-Syp_P1 cluster with 220804-Picc_P2 cls 12/24 = 0.50TID2 214/420 = 0.51
220804-Picc_P2 cluster with 220804-Syp_P1 cls 12/99 = 0.12TID2 464/1127 = 0.41

export csv table
export paraview file


In [7]:
eps_range = [1e-8, 2e-8, 3e-8, 4e-8, 5e-8, 6e-8, 7e-8, 8e-8]
for label in npy_cwash:
    for afile in npy_cwash[label]:
        processFile(afile, eps_range=eps_range)

2022-09-23 14:18:52,765 [INFO] **** trim_min_localizations ****
220804_VGLUT1_P1 Removed 127/336 tracks with less than 3 localizations
220804_VGLUT1_P2 Removed 115/303 tracks with less than 3 localizations

2022-09-23 14:18:55,151 [INFO] **** cluster_tid ****
220804_VGLUT1_P1 MIN_SPLIT_LOCALIZATION: 6, sd_limit: 10.93 nm
Processed TID: 42 / 209, Total tracks TID2: 248
220804_VGLUT1_P2 MIN_SPLIT_LOCALIZATION: 6, sd_limit: 10.93 nm
Processed TID: 41 / 188, Total tracks TID2: 224

2022-09-23 14:18:55,154 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis//Multiwash/VGLUT1_VGLUT1/220804_VGLUT1_ROI01_First\220804_VGLUT1_ROI01_First.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 1e-08
DBCLUSTER_EPS_ALL 1e-08
DBCLUSTER_EPS_MERGED_ALL 1e-08
DBCLUSTER_EPS_MERGED_MEAS 1e-08
ref_error {'se_xy': 0.0, 'se_z': 0.0, 'std_xy': 0.0, 'std_z': 0.0}
logger <Logger processlocalizations (INFO)>

2022-09-23 14:18:55,212 [INFO] 

2022-09-23 14:18:57,604 [INFO] **** cluster_all ****
220804_VGLUT1_P1, Method: dbscan, eps: 40.00 nm
Total tracks TID2: 247, total cluster all: 61
220804_VGLUT1_P2, Method: dbscan, eps: 40.00 nm
Total tracks TID2: 223, total cluster all: 79

2022-09-23 14:18:57,630 [INFO] **** cluster_all_intersect ****
220804_VGLUT1_P1 cluster with 220804_VGLUT1_P2 cls 23/60 = 0.38TID2 190/247 = 0.77
220804_VGLUT1_P2 cluster with 220804_VGLUT1_P1 cls 23/79 = 0.29TID2 143/223 = 0.64

export csv table
export paraview file
2022-09-23 14:18:57,950 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis//Multiwash/VGLUT1_VGLUT1/220804_VGLUT1_ROI01_First\220804_VGLUT1_ROI01_First.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 5e-08
DBCLUSTER_EPS_ALL 5e-08
DBCLUSTER_EPS_MERGED_ALL 5e-08
DBCLUSTER_EPS_MERGED_MEAS 5e-08
ref_error {'se_xy': 0.0, 'se_z': 0.0, 'std_xy': 0.0, 'std_z': 0.0}
logger <Logger processlocalizations (INFO)>

2022

2022-09-23 14:19:00,845 [INFO] **** cluster_all ****
220804_VGLUT1_P1, Method: dbscan, eps: 80.00 nm
Total tracks TID2: 247, total cluster all: 35
220804_VGLUT1_P2, Method: dbscan, eps: 80.00 nm
Total tracks TID2: 223, total cluster all: 43

2022-09-23 14:19:00,865 [INFO] **** cluster_all_intersect ****
220804_VGLUT1_P1 cluster with 220804_VGLUT1_P2 cls 17/34 = 0.50TID2 223/247 = 0.90
220804_VGLUT1_P2 cluster with 220804_VGLUT1_P1 cls 17/43 = 0.40TID2 180/223 = 0.81

export csv table
export paraview file
2022-09-23 14:19:01,459 [INFO] **** trim_min_localizations ****
220811_VGLUT1_P1 Removed 927/2692 tracks with less than 3 localizations
220811_VGLUT1_P2 Removed 738/1813 tracks with less than 3 localizations

2022-09-23 14:20:07,922 [INFO] **** cluster_tid ****
220811_VGLUT1_P1 MIN_SPLIT_LOCALIZATION: 6, sd_limit: 11.11 nm
Processed TID: 354 / 1765, Total tracks TID2: 2021
220811_VGLUT1_P2 MIN_SPLIT_LOCALIZATION: 6, sd_limit: 11.11 nm
Processed TID: 225 / 1075, Total tracks TID2: 1234


2022-09-23 14:20:37,881 [INFO] **** cluster_all ****
220811_VGLUT1_P1, Method: dbscan, eps: 40.00 nm
Total tracks TID2: 2020, total cluster all: 346
220811_VGLUT1_P2, Method: dbscan, eps: 40.00 nm
Total tracks TID2: 1233, total cluster all: 282

2022-09-23 14:20:38,041 [INFO] **** cluster_all_intersect ****
220811_VGLUT1_P1 cluster with 220811_VGLUT1_P2 cls 72/345 = 0.21TID2 1180/2020 = 0.58
220811_VGLUT1_P2 cluster with 220811_VGLUT1_P1 cls 72/281 = 0.26TID2 705/1233 = 0.57

2022-09-23 14:20:40,935 [INFO] **** cluster_meas ****
220811_VGLUT1_P1, Method dbscan, eps: 40.00 nm
Total tracks TID2: 2020, total clusters meas: 395
220811_VGLUT1_P2, Method dbscan, eps: 40.00 nm
Total tracks TID2: 1233, total clusters meas: 363

2022-09-23 14:20:42,414 [INFO] **** cluster_all ****
220811_VGLUT1_P1, Method: dbscan, eps: 40.00 nm
Total tracks TID2: 2020, total cluster all: 346
220811_VGLUT1_P2, Method: dbscan, eps: 40.00 nm
Total tracks TID2: 1233, total cluster all: 282

2022-09-23 14:20:42,581 

2022-09-23 14:21:22,356 [INFO] **** cluster_meas ****
220811_VGLUT1_P1, Method dbscan, eps: 80.00 nm
Total tracks TID2: 2020, total clusters meas: 146
220811_VGLUT1_P2, Method dbscan, eps: 80.00 nm
Total tracks TID2: 1233, total clusters meas: 148

2022-09-23 14:21:24,726 [INFO] **** cluster_all ****
220811_VGLUT1_P1, Method: dbscan, eps: 80.00 nm
Total tracks TID2: 2020, total cluster all: 106
220811_VGLUT1_P2, Method: dbscan, eps: 80.00 nm
Total tracks TID2: 1233, total cluster all: 75

2022-09-23 14:21:24,887 [INFO] **** cluster_all_intersect ****
220811_VGLUT1_P1 cluster with 220811_VGLUT1_P2 cls 22/105 = 0.21TID2 65/2020 = 0.03
220811_VGLUT1_P2 cluster with 220811_VGLUT1_P1 cls 22/74 = 0.30TID2 67/1233 = 0.05

2022-09-23 14:21:28,148 [INFO] **** cluster_meas ****
220811_VGLUT1_P1, Method dbscan, eps: 80.00 nm
Total tracks TID2: 2020, total clusters meas: 146
220811_VGLUT1_P2, Method dbscan, eps: 80.00 nm
Total tracks TID2: 1233, total clusters meas: 148

2022-09-23 14:21:30,485 [I